In [62]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import tqdm
import re
import string
import math


In [63]:
df = pd.read_csv(r'C:\Users\Shaktiy\Desktop\PyML\PythonPracticeTextFiles\Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [64]:
df_copy = df.iloc[:500,:][['Id', 'Text']]
df_copy.shape

(500, 2)

In [66]:
for i in df_copy['Text']:
    if 'peanut' in i.lower():
        print(i)
        print("======================================================================")

Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
This wasn't in stock the last time I looked. I had to go to the Vermont Country Store in Weston to find it along with a jaw harp, Cranberry Horseradish Sauce, Fartless Black Bean Salsa, Apple Cider Jelly, Newton's Cradle Art in Motion and the staple Vermont Maple Syrup.<br /><br />Back to the Ass Kickin Peanuts. They are hot. They will activate the perspiration glands behind your ears and under your arms. It requires a beverage as advertised, a glass of very cold milk, and a box of Kleenex since it will make your nose run. They look like ordinary peanuts which is already giving me ideas for work. I suspect that some people have been hitting my goodies in my absence, especially my colleague Greg. I'm going to take this to work at earliest opportunity and empty the contents of this can into an ordina

### Data Cleaning 

In [67]:
clean_txt = re.compile('<.*?>')
def clean_html(text):
    cleantext = re.sub(clean_txt, ' ', text)
    return cleantext

In [68]:
def DataCleaning(corpus):

    ps = PorterStemmer()
    final = []

    for line in tqdm.tqdm(corpus):
        line = clean_html(line)
        line = line.lower() # Converts text to lower case
        line = clean_html(line) # remove html
        line = ' '.join([wrd for wrd in line.split() if wrd not in stopwords.words('english')]) #remove stopwords
        line = ' '.join([wrd for wrd in line.split() if wrd.isalnum()]) # remove punc and other words that are n ot alpha numeric
        line =  ' '.join([ps.stem(wrd) for wrd in line.split()])
        final.append(line)
    return final

In [89]:
corpus = DataCleaning(df_copy['Text'])


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 56.45it/s]


### Getting Unique Words  

In [70]:
def unique_words(corpus):
    unique_words = []
    for i in corpus:
        unique_words.extend(i.split())
    unique_words = set(unique_words)
    return(unique_words)

### IDF Calculation 

In [71]:
def IDF(corpus):
    idf_dict={}
    N=len(corpus)
    for i in unque_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1
        idf_dict[i]=(math.log((N)/(count)))
    return idf_dict 


In [72]:
idf_dict = IDF(corpus)

In [73]:
idf_dict['tasti']

3.912023005428146

In [74]:
unque_words = unique_words(corpus)

### TF-IDF function [Calculation for given corpus and unique word ]

In [75]:
def TF_IDF(corpus, unque_words):
    final = []
    for sen in corpus:
        inter = []
        for unq in unque_words:
            cnt = sen.split().count(unq)
            tf_idf = cnt*idf_dict[unq]
            inter.append(tf_idf)
        final.append(inter)
    return(final)

### TF-IDF function [Given new review in corpus unique word] 

In [76]:
def TF_IDf_Review(corpus, unque_words, rev):
    final = []
    for wrd in unque_words:
        if wrd in rev.split():
            tfidf = rev.count(wrd)*idf_dict[wrd]
            final.append(tfidf)   
        else:
            final.append(0)
    return final    

### Dataframe for TF-IDF  

In [77]:
uq = unique_words(corpus)
Final = TF_IDF(corpus, uq)


In [78]:
f = pd.DataFrame(Final, columns = uq)

In [79]:
f.head()

,savora,sign,eat,regularli,content,healthiest,act,albanes,bake,nescaf,...,fat,hotlin,insid,thin,around,popper,amo,role,canning,solomon
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.442019,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### Vector using TF-IDF Vectorizer to validate if values calculated from above python code from scratch is equal to the one we got directly using tfidf

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [75]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [76]:
X.shape, len(vectorizer.get_feature_names())

((500, 2477), 2477)

In [77]:
(10,1000, 0.32), (9,500,0.53)

((10, 1000, 0.32), (9, 500, 0.53))

In [78]:
d_sk = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [79]:
d_sk.head()

,10,100,1000,100ml,10lb,11,110,12,13,1300watt,...,york,young,younger,youngest,yr,yucki,yummi,zen,zest,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.175297,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.306047,0.0,0.0,0.0


In [80]:
(d_sk['young']==f['young']).sum()

499

### To see if two reviews are same 

In [80]:
SampleReview = "Peanut butter is very cheap and it is very tasty"

f = DataCleaning([SampleReview])

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 50.00it/s]


In [81]:
f

['peanut butter cheap tasti']

In [82]:
rev = f[0]
rev

'peanut butter cheap tasti'

In [83]:
unq_wrds = unique_words(corpus)

### Cosine Similarity to find out similar reviews to a sample review 

In [84]:
def CosineSimilarity(a, vectorB, corpus):
    coslist = []
    for i in a:
        vectorA = i
        cos = np.dot(vectorA,vectorB)/(np.sqrt(np.dot(vectorA,vectorA))*np.sqrt(np.dot(vectorB,vectorB)))
        coslist.append(cos)
    similarReview = corpus[np.argmax(coslist)]    
    return similarReview

In [85]:
a = TF_IDF(corpus, unque_words)
vectorB = TF_IDf_Review(corpus, unque_words, rev)
CosineSimilarity(a, vectorB, corpus)

'natur valley nut lover varieti pack although sure peanut butter love roast roast pecan peanut defin buy seller quick ship'

### Euclidean Distance to find out similar reviews to sample review 

In [87]:
def EuclideanSimilarity(a, vectorB, corpus):
    euclist = []
    for i in a:
        vectorA = i
        euclideandist = np.linalg.norm(np.array(vectorA)-np.array(vectorB))
        euclist.append(euclideandist)
    similarReview = corpus[np.argmin(euclist)]    
    return similarReview    

In [88]:
a = TF_IDF(corpus, unque_words)
vectorB = TF_IDf_Review(corpus, unque_words, rev)
EuclideanSimilarity(a, vectorB, corpus)

'would go way buy'